# Import bibliotek i danych

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
import numpy as np
import pickle
import typing
warnings.filterwarnings("ignore")
SEED = 17

# Ładowanie danych

In [2]:
data=pd.read_csv("../data/data_eda.csv")
data=data.drop(columns=['Unnamed: 0'])
features=data.columns.tolist()
features.remove('stars')
target='stars'
data

,pages,stars,reviews,series,mix,character,plot,funny,lighthearted,emotional,...,author_stars,Fiction,Nonfiction,Literary,Fantasy,Crime,Social,Children,Romans,Realism
0,273,4.00,2017,0,0.44,0.51,0.02,0.27,0.37,0.91,...,4.305000,1,1,0,0,0,1,0,1,1
1,302,3.78,7330,0,0.39,0.42,0.17,0.03,0.01,0.18,...,3.670000,1,0,0,0,1,0,0,0,0
2,400,4.15,16761,0,0.51,0.39,0.08,0.02,0.01,0.88,...,0.000000,1,0,1,0,0,0,0,0,0
3,459,4.16,2128,1,0.48,0.10,0.40,0.04,0.02,0.07,...,0.000000,1,0,0,1,0,0,0,0,0
4,160,3.65,6634,1,0.28,0.16,0.54,0.92,0.73,0.00,...,4.115000,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,432,4.15,30643,0,0.48,0.05,0.46,0.00,0.00,0.40,...,3.856667,1,0,0,1,1,0,0,0,0
6926,352,3.62,1058,0,0.55,0.13,0.30,0.15,0.10,0.25,...,3.700000,1,0,0,1,0,1,0,0,0
6927,535,3.88,30975,1,0.45,0.08,0.45,0.14,0.19,0.31,...,3.870000,1,0,0,1,0,0,1,0,0
6928,472,3.88,5914,1,0.64,0.12,0.22,0.07,0.00,0.36,...,3.660000,1,0,0,1,0,0,1,0,0


$\text{Podział danych na zbiór treningowy i testowy}$

In [3]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=SEED)

In [4]:
def perform_cv(X: pd.DataFrame, y: pd.Series, algorithm: typing.Any, cv: typing.Any = KFold(n_splits=5, shuffle=True, random_state=SEED), metric: typing.Any = mean_squared_error) -> typing.List[float]:
    """
    Perform cross-validation and return list of scores
    
    Args:
        X (pd.DataFrame): input data
        y (pd.Series): target data
        algorithm (typing.Any): algorithm to use for training and prediction
        cv (typing.Any): cross-validation strategy
        metric (typing.Any): metric to use for evaluation
    
    Returns:
        typing.List[float]: list of scores in order: train_scores, validation_scores
    """
    train_scores = []
    validation_scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        algorithm.fit(X_train, y_train)
        y_train_pred = algorithm.predict(X_train)
        y_val_pred = algorithm.predict(X_val)
        train_scores.append(metric(y_train, y_train_pred, squared=False))
        validation_scores.append(metric(y_val, y_val_pred, squared=False))
    return np.mean(train_scores), np.mean(validation_scores)

def evaluation(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series, algorithm: typing.Any, metric: typing.Any = mean_squared_error) -> typing.Tuple[float, float, np.ndarray]:
    """
    Train the algorithm on the train data and evaluate on the train and test data
    
    Args:
        X_train (pd.DataFrame): input train data
        y_train (pd.Series): target train data
        X_test (pd.DataFrame): input test data
        y_test (pd.Series): target test data
        algorithm (typing.Any): algorithm to use for training and prediction
        metric (typing.Any): metric to use for evaluation
    
    Returns:
        typing.Tuple[float, float, np.ndarray]: train_score, test_score, predictions on test data
    """
    algorithm.fit(X_train, y_train)
    y_train_pred = algorithm.predict(X_train)
    y_test_pred = algorithm.predict(X_test)
    train_results = metric(y_train, y_train_pred, squared=False)
    test_results = metric(y_test, y_test_pred, squared=False)
    return train_results, test_results, y_test_pred

### Najbardziej podstawowy model (bez feature engineeringu):

In [5]:
wzor = 'stars~' + '+'.join(features)
mod = smf.ols(formula = wzor, data = train_data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.464
Model:                            OLS   Adj. R-squared:                  0.461
Method:                 Least Squares   F-statistic:                     153.9
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        19:38:29   Log-Likelihood:                 512.46
No. Observations:                5544   AIC:                            -960.9
Df Residuals:                    5512   BIC:                            -749.1
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.0217      0.036     84.278      0.000       2.951       3.092
pages         -2.24e-05   2.22e-05     -1.008      0.314    -6.6e-05    2.12e-05
reviews       1.023e-06   1.23e-07      8.348      0.000    7.83e-07    1.26e-06
series           0.0708      0.008      8.948      0.000       0.055       0.086
mix              0.0731      0.033      2.238      0.025       0.009       0.137
character       -0.0894      0.027     -3.357      0.001      -0.142      -0.037
plot            -0.1755      0.038     -4.609      0.000      -0.250      -0.101
funny            0.4076      0.021     19.274      0.000       0.366       0.449
lighthearted    -0.0356      0.036     -0.987      0.324      -0.106       0.035
emotional        0.3759      0.028     13.635      0.000       0.322       0.430
hopeful          0.2907      0.039      7.535      0.000       0.215       0.366
inspiring        0.2729      0.036      7.544      0.000       0.202       0.344
relaxing         0.7907      0.075     10.521      0.000       0.643       0.938
tense            0.2778      0.033      8.393      0.000       0.213       0.343
sad              0.2105      0.029      7.289      0.000       0.154       0.267
reflective      -0.0111      0.028     -0.403      0.687      -0.065       0.043
adventurous      0.1818      0.019      9.746      0.000       0.145       0.218
challenging      0.4286      0.030     14.496      0.000       0.371       0.487
informative      0.4146      0.025     16.538      0.000       0.365       0.464
mysterious       0.1753      0.020      8.930      0.000       0.137       0.214
dark            -0.0036      0.022     -0.169      0.866      -0.046       0.039
author_count     0.0025      0.000      5.206      0.000       0.002       0.003
author_stars     0.0187      0.002     10.596      0.000       0.015       0.022
Fiction          0.0786      0.032      2.461      0.014       0.016       0.141
Nonfiction       0.1668      0.017      9.681      0.000       0.133       0.201
Literary         0.0280      0.008      3.437      0.001       0.012       0.044
Fantasy          0.0242      0.010      2.364      0.018       0.004       0.044
Crime            0.0053      0.012      0.436      0.663      -0.018       0.029
Social           0.0530      0.009      6.132      0.000       0.036       0.070
Children        -0.0269      0.009     -3.148      0.002      -0.044      -0.010
Romans          -0.0634      0.010     -6.224      0.000      -0.083      -0.043
Realism         -0.0499      0.009     -5.621      0.000      -0.067      -0.033
==============================================================================
Omnibus:                      206.478   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              306.045
Skew:      

In [6]:
model = LinearRegression(fit_intercept=True)
train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
print("Train RMSE:", train_scores)
print("Validation RMSE:", validation_scores)

Train RMSE: 0.2204369364743511
Validation RMSE: 0.2220773168372384


## Model z interakcjami

In [7]:
data_interactions=pd.read_csv("../data/data_add.csv")
data_interactions=data_interactions.drop(columns=['Unnamed: 0'])
features_interactions=data_interactions.columns.tolist()
features_interactions.remove('stars')
train_data_interactions, test_data_interactions = train_test_split(data_interactions, test_size=0.2, random_state=SEED)
wzor = 'stars~' + '+'.join(features_interactions)
mod = smf.ols(formula = wzor, data = train_data_interactions)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     90.01
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        19:38:29   Log-Likelihood:                 753.11
No. Observations:                5544   AIC:                            -1378.
Df Residuals:                    5480   BIC:                            -954.5
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.0548      0.056     54.741      0.000       2.945       3.164
index_0                 -1.187e-05   1.46e-06     -8.137      0.000   -1.47e-05   -9.01e-06
pages                   -1.542e-05   2.17e-05     -0.710      0.478    -5.8e-05    2.71e-05
reviews                  7.376e-07    1.2e-07      6.162      0.000    5.03e-07    9.72e-07
series                      0.0021      0.022      0.094      0.925      -0.042       0.046
mix                         0.0449      0.022      2.018      0.044       0.001       0.088
character                  -0.0488      0.013     -3.774      0.000      -0.074      -0.023
plot                       -0.0608      0.020     -3.112      0.002      -0.099      -0.023
funny                       0.4203      0.021     19.939      0.000       0.379       0.462
lighthearted               -0.0195      0.036     -0.536      0.592      -0.091       0.052
emotional                   0.4435      0.060      7.430      0.000       0.327       0.561
hopeful                     0.0860      0.124      0.695      0.487      -0.157       0.329
inspiring                   0.3943      0.060      6.617      0.000       0.277       0.511
relaxing                    0.8472      0.074     11.487      0.000       0.703       0.992
tense                       0.0362      0.130      0.278      0.781      -0.219       0.291
sad                         0.2220      0.076      2.927      0.003       0.073       0.371
reflective                 -0.2326      0.059     -3.929      0.000      -0.349      -0.117
adventurous                -0.0065      0.065     -0.100      0.921      -0.135       0.121
challenging                 1.6755      0.146     11.473      0.000       1.389       1.962
informative                 0.3665      0.028     12.879      0.000       0.311       0.422
mysterious                  0.4816      0.126      3.837      0.000       0.236       0.728
dark                       -0.1860      0.068     -2.748      0.006      -0.319      -0.053
author_count                0.0022      0.000      4.612      0.000       0.001       0.003
author_stars               -0.0013      0.009     -0.139      0.889      -0.020       0.017
Fiction                     0.0179      0.057      0.314      0.753      -0.094       0.130
Nonfiction                  0.2561      0.037      6.917      0.000       0.184       0.329
Literary                    0.0920      0.023      4.026      0.000       0.047       0.137
Fantasy                    -0.0153      0.025     -0.614      0.539      -0.064       0.034
Crime                      -0.0124      0.027     -0.465      0.642      -0.065       0.040
Social                      0.0065      0.013      0.509      0.611      -0.019       0.032
Children                   -0.0324      0.008  

In [8]:
model = LinearRegression(fit_intercept=True)
train_scores, validation_scores = perform_cv(train_data_interactions[features_interactions], train_data_interactions[target], model)
print("Train RMSE:", train_scores)
print("Validation RMSE:", validation_scores)

Train RMSE: 0.21091275251763042
Validation RMSE: 0.21410239828253755


## Model z transformacjami zmiennych

In [9]:
data_transformations=pd.read_csv("../data/data_fe.csv")
data_transformations=data_transformations.drop(columns=['Unnamed: 0'])
features_transformations=data_transformations.columns.tolist()
features_transformations.remove('stars')
train_data_transformations, test_data_transformations = train_test_split(data_transformations, test_size=0.2, random_state=SEED)
wzor = 'stars~' + '+'.join(features_transformations)
mod = smf.ols(formula = wzor, data = train_data_transformations)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     98.76
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        19:38:29   Log-Likelihood:                 254.48
No. Observations:                5544   AIC:                            -429.0
Df Residuals:                    5504   BIC:                            -164.1
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            2.5155      0.116     21.716      0.000       2.288       2.743
Crime                                0.0159      0.013      1.268      0.205      -0.009       0.040
Romans                              -0.1002      0.010    -10.071      0.000      -0.120      -0.081
sad_mm                               0.2819      0.025     11.277      0.000       0.233       0.331
reviews_boxcox                      -0.3657      0.037     -9.832      0.000      -0.439      -0.293
reflective_boxcox                    0.0244      0.007      3.341      0.001       0.010       0.039
mysterious_rs                        0.0424      0.012      3.654      0.000       0.020       0.065
inspiring_log                        0.0111      0.004      2.914      0.004       0.004       0.019
informative_boxcox                   0.0024      0.001      1.860      0.063      -0.000       0.005
emotional_boxcox                     0.0133      0.019      0.711      0.477      -0.023       0.050
challenging_mm                       0.4429      0.111      3.990      0.000       0.225       0.661
author_stars                         0.0034      0.003      1.087      0.277      -0.003       0.010
Fantasy_author_stars                 0.0045      0.004      1.227      0.220      -0.003       0.012
Fantasy_mix_qt                       0.0178      0.020      0.870      0.385      -0.022       0.058
Fiction_adventurous_qt               0.1298      0.022      5.965      0.000       0.087       0.172
Fiction_author_count_ss              0.0201      0.004      5.543      0.000       0.013       0.027
Fiction_challenging_mm              -0.1187      0.109     -1.093      0.274      -0.331       0.094
Fiction_character_ss                -0.0519      0.006     -8.502      0.000      -0.064      -0.040
Fiction_emotional_boxcox            -0.1172      0.022     -5.223      0.000      -0.161      -0.073
Fiction_hopeful_rs                   0.1084      0.009     12.319      0.000       0.091       0.126
Fiction_mix_qt                       0.1775      0.024      7.425      0.000       0.131       0.224
Fiction_pages_qt                    -0.0085      0.014     -0.593      0.553      -0.037       0.020
Fiction_plot_qt                     -0.3081      0.025    -12.095      0.000      -0.358      -0.258
Literary_author_stars                0.0063      0.003      1.880      0.060      -0.000       0.013
Literary_emotional_boxcox            0.0074      0.016      0.454      0.650      -0.025       0.039
Literary_hopeful_rs                  0.0247      0.010      2.354      0.019       0.004       0.045
Literary_inspiring_log              -0.0084      0.005     -1.841      0.066      -0.017       0.001
Literary_mysterious_rs              -0.0188      0.01

In [10]:
model = LinearRegression(fit_intercept=True)
train_scores, validation_scores = perform_cv(train_data_transformations[features_transformations], train_data_transformations[target], model)
print("Train RMSE:", train_scores)
print("Validation RMSE:", validation_scores)

Train RMSE: 0.23086218185156407
Validation RMSE: 0.23336460790373134


Najlepszy wynik dla modelu z interkacjami bez transformacji, ale nadal dużo zmiennych wyszło jako nieistotne statystyczne, nawet te o dużej wartości MI, dlatego przeprowadzimy analizę VIF współliniowości

In [11]:
# the independent variables set 
X = train_data_interactions[features_interactions]
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))] 
vif_data.sort_values(by=["VIF"], ascending=False).head(50)

,feature,VIF
40,Fiction_character,inf
45,Fiction_mix,inf
4,mix,inf
5,character,inf
6,plot,inf
47,Fiction_plot,inf
46,Fiction_mysterious,344.819537
19,mysterious,344.498437
50,Fiction_tense,258.057208
13,tense,249.380952


Na początek usuniemy z modelu zmienne: mix, character, plot, Fiction_mysterious, Fiction_tense, Fiction_adventurous, Fiction_dark, Fiction_emotional, Fiction_challenging, Fiction_hopeful, Fiction_author_stars, Fiction_sad, Fiction_reflective, Nonfiction_challenging, series_mix, Fiction, Fanstasy_mix i Literary_sad

In [12]:
features_to_remove = ["mix", "character", "plot", "Fiction_mysterious", "Fiction_tense", "Fiction_adventurous", "Fiction_dark", "Fiction_emotional", "Fiction_challenging", "Fiction_hopeful", "Fiction_author_stars", "Fiction_sad", "Fiction_reflective", "Nonfiction_challenging", "series_mix", "Fiction", "Fantasy_mix", "Literary_sad"]
for feature in features_to_remove:
    features_interactions.remove(feature)

In [13]:
wzor = 'stars~' + '+'.join(features_interactions)
mod = smf.ols(formula = wzor, data = train_data_interactions)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.491
Model:                            OLS   Adj. R-squared:                  0.487
Method:                 Least Squares   F-statistic:                     110.5
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        19:38:32   Log-Likelihood:                 657.04
No. Observations:                5544   AIC:                            -1216.
Df Residuals:                    5495   BIC:                            -891.7
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.0893      0.035     89.400      0.000       3.022       3.157
index_0                 -1.223e-05   1.48e-06     -8.264      0.000   -1.51e-05   -9.33e-06
pages                   -2.326e-05   2.19e-05     -1.063      0.288   -6.61e-05    1.96e-05
reviews                  7.963e-07   1.21e-07      6.567      0.000    5.59e-07    1.03e-06
series                      0.0412      0.020      2.088      0.037       0.003       0.080
funny                       0.3933      0.021     18.793      0.000       0.352       0.434
lighthearted               -0.0076      0.036     -0.213      0.832      -0.078       0.063
emotional                   0.3710      0.031     11.987      0.000       0.310       0.432
hopeful                     0.2893      0.041      7.032      0.000       0.209       0.370
inspiring                   0.2474      0.049      5.092      0.000       0.152       0.343
relaxing                    0.7824      0.074     10.567      0.000       0.637       0.928
tense                       0.2914      0.033      8.859      0.000       0.227       0.356
sad                         0.1910      0.029      6.645      0.000       0.135       0.247
reflective                 -0.0250      0.028     -0.909      0.364      -0.079       0.029
adventurous                 0.0749      0.023      3.270      0.001       0.030       0.120
challenging                 0.4226      0.029     14.359      0.000       0.365       0.480
informative                 0.4376      0.025     17.463      0.000       0.388       0.487
mysterious                  0.1813      0.023      7.983      0.000       0.137       0.226
dark                        0.0346      0.021      1.621      0.105      -0.007       0.076
author_count                0.0022      0.000      4.515      0.000       0.001       0.003
author_stars                0.0111      0.003      3.242      0.001       0.004       0.018
Nonfiction                  0.1958      0.021      9.442      0.000       0.155       0.237
Literary                    0.0946      0.023      4.151      0.000       0.050       0.139
Fantasy                    -0.0175      0.019     -0.928      0.353      -0.054       0.019
Crime                      -0.0422      0.025     -1.687      0.092      -0.091       0.007
Social                      0.0210      0.013      1.655      0.098      -0.004       0.046
Children                   -0.0361      0.009     -4.237      0.000      -0.053      -0.019
Romans                     -0.0378      0.015     -2.443      0.015      -0.068      -0.007
Realism                    -0.0454      0.009     -5.104      0.000      -0.063      -0.028
Crime_mix                   0.1118      0.052      2.164      0.030       0.011       0.213
Fantasy_adventurous         0.1171      0.028  

Pozostały zmienne nieistotne statystycznie: Fantasy_author_stars, Fantasy_inspiring, Nonfiction_author_stars, Romans_author_stars oraz reflective, które mają niską wartość VIF.

In [14]:
features_to_remove = ["Fantasy_author_stars", "Fantasy_inspiring", "Nonfiction_author_stars", "Romans_author_stars", "reflective"]
for feature in features_to_remove:
    features_interactions.remove(feature)

Również: lighthearted, która mają stosunkową wysoką wartość VIF.
Z EDA wiemy, że zmienna lighthearted jest skorelowana z funny oraz relaxing, więc spróbujemy utworzyć nową zmienną.

In [15]:
train_data_interactions['lighthearted_relaxing']=train_data_interactions['lighthearted']+train_data_interactions['relaxing']
train_data_interactions['lighthearted_funny']=train_data_interactions['lighthearted']+train_data_interactions['funny']
features_to_add = ["lighthearted_funny", "lighthearted_relaxing"]
for feature in features_to_add:
    features_interactions.append(feature)

In [16]:
wzor = 'stars~' + '+'.join(features_interactions)
mod = smf.ols(formula = wzor, data = train_data_interactions)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  stars   R-squared:                       0.491
Model:                            OLS   Adj. R-squared:                  0.487
Method:                 Least Squares   F-statistic:                     123.4
Date:                Sat, 11 May 2024   Prob (F-statistic):               0.00
Time:                        19:38:32   Log-Likelihood:                 656.18
No. Observations:                5544   AIC:                            -1224.
Df Residuals:                    5500   BIC:                            -933.1
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 3.0791      0.031    100.632      0.000       3.019       3.139
index_0               -1.221e-05   1.48e-06     -8.257      0.000   -1.51e-05   -9.31e-06
pages                 -2.215e-05   2.18e-05     -1.014      0.311    -6.5e-05    2.07e-05
reviews                7.955e-07   1.21e-07      6.571      0.000    5.58e-07    1.03e-06
series                    0.0448      0.019      2.323      0.020       0.007       0.083
funny                     0.3396      0.025     13.849      0.000       0.291       0.388
lighthearted             -0.2875      0.033     -8.600      0.000      -0.353      -0.222
emotional                 0.3679      0.031     11.953      0.000       0.308       0.428
hopeful                   0.2933      0.041      7.178      0.000       0.213       0.373
inspiring                 0.2422      0.048      5.041      0.000       0.148       0.336
relaxing                  0.5291      0.052     10.261      0.000       0.428       0.630
tense                     0.3015      0.031      9.706      0.000       0.241       0.362
sad                       0.1920      0.029      6.692      0.000       0.136       0.248
adventurous               0.0783      0.023      3.470      0.001       0.034       0.123
challenging               0.4143      0.028     14.981      0.000       0.360       0.469
informative               0.4404      0.025     17.848      0.000       0.392       0.489
mysterious                0.1826      0.023      8.065      0.000       0.138       0.227
dark                      0.0376      0.021      1.805      0.071      -0.003       0.078
author_count              0.0022      0.000      4.571      0.000       0.001       0.003
author_stars              0.0094      0.003      3.366      0.001       0.004       0.015
Nonfiction                0.1932      0.018     10.861      0.000       0.158       0.228
Literary                  0.0898      0.022      4.018      0.000       0.046       0.134
Fantasy                  -0.0254      0.015     -1.702      0.089      -0.055       0.004
Crime                    -0.0404      0.025     -1.624      0.104      -0.089       0.008
Social                    0.0218      0.012      1.744      0.081      -0.003       0.046
Children                 -0.0353      0.008     -4.172      0.000      -0.052      -0.019
Romans                   -0.0419      0.010     -4.262      0.000      -0.061      -0.023
Realism                  -0.0461      0.009     -5.199      0.000      -0.063      -0.029
Crime_mix                 0.1090      0.051      2.119      0.034       0.008       0.210
Fantasy_adventurous       0.1196      0.027      4.372      0.000       0.066       0.173
Fiction_character        -0.0789      0.021     -3.748      0.000      -0.120      -0.038
Fiction_inspiring      

Reszta zmiennych w modelu jest istotna statystycznie

Większość zmiennych wpływa pozytywnie na zmienną objaśnianą oprócz: pages, Fantasy, Crime, Children, Romans, Realism, Fiction_character, Fiction_plot, Literary_inspiring, Literary_mix, Nonfiction_hopeful, series_emotional i lighthearted_funny.
Jednak zmienna Fantasy jest również obecna w interakcji z adventurous, a zmienna Crime w interkacji z mix.

Można wyciągnąc wniosek, że dobrze są oceniane ksiązki Fantasy jedynie, gdy mają wysoki wskaźnik przygodowości, a Crime, gdy mają mix postaci i fabuły. Gorzej są oceniane książki z genre fiction jedynie skupione na postaciach lub fabule, a nie mixie. Dodatkowo kategorie Children, Romans i Realism mają średnio niższe oceny. Również mniej lubione są książki z kategorii Literary, które są inspirujące lub mają mix fabułu i postaci, także książki Nonfiction, które są pełne nadziei i emocjonalne serie.
Zaskakująco ludzie również gorzej oceniają książki, które są śmieszne jeśli są beztroskie, możliwe, że preferowane są książki, które mają tzw. dark humor.

## Zapisanie modelu

Nie usuwamy zmiennych z modelu, gdyż w tym przypadku OLS zmniejszyłoby to zdolność predykcyjną.

In [17]:
data_interactions=pd.read_csv("../data/data_add.csv")
data_interactions=data_interactions.drop(columns=['Unnamed: 0'])
features_interactions=data_interactions.columns.tolist()
features_interactions.remove('stars')
data_interactions['lighthearted_relaxing']=data_interactions['lighthearted']+data_interactions['relaxing']
data_interactions['lighthearted_funny']=data_interactions['lighthearted']+data_interactions['funny']
train_data_interactions, test_data_interactions = train_test_split(data_interactions, test_size=0.2, random_state=SEED)
test_indices = test_data_interactions.index
features_interactions.append('lighthearted_funny')
features_interactions.append('lighthearted_relaxing')
target='stars'
#Ewaluacja modelu
model = LinearRegression(fit_intercept=True)
train_results, test_results, y_test_pred = evaluation(train_data_interactions[features_interactions], train_data_interactions[target], test_data_interactions[features_interactions], test_data_interactions[target], model)
print("Train RMSE: {}".format(round(train_results, 5)))
print("Test RMSE: {}".format(round(test_results, 5)))

#Zapisanie modelu
modelOLS = {
    "name": "OLS",
    "trainResults": train_results,
    "testResults": test_results,
    "predictions": y_test_pred,
    "indices": test_indices,
}

with open("../data/model_OLS.p", "wb") as fp:
    pickle.dump(modelOLS, fp)

Train RMSE: 0.21124
Test RMSE: 0.21884


## Podsumowanie

$\text{Najlepsze wyniki walidacji krzyżowej uzyskano dla modelu bazującego na danych z interakcjami, bez transformacji zmiennych.}$<p>
$\text{Wyniki na zbiorze treningowym (RMSE): 0.21124}$<p>
$\text{Wyniki na zbiorze testowym (RMSE): 0.21884}$<p>